In [1]:
from selenium import webdriver

In [2]:
driver = webdriver.Chrome()

In [8]:
import requests
import time
from bs4 import BeautifulSoup
import forecastio

- api json을 사용: 네이버 주식 데이터 크롤링
- api json을 사용, forecastio: dark sky api로 날씨 데이터를 수집
- bs4: 네이버 실시간 키워드, 다음 실시간 키워드 크롤링
- web file url로 파일을 다운로드 받는 방법

In [57]:
from pandas.io.json import json_normalize

##### 네이버 주식 데이터 가져오기

In [10]:
# url 만들기
def make_url(pageSize=10, page=1):
    return "http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize=" + str(pageSize) + "&page=" + str(page)
make_url()

'http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize=10&page=1'

In [11]:
import pandas as pd

In [59]:
# 데이터 가져와서 pandas의 dataframe으로 만드는 함수
def get_data(url):
    response = requests.get(url) #여기서 get 쓰면 get 방식으로 요청, post로 요청할수도 있음. response 객체로 만들어서 저장.
    json_info = response.json() #response 안에 있는 text데이터를 json 방식으로 받음.
    companies = json_info["result"]["itemList"] #companies = 종목
    return json_normalize(companies)
    
#    df = pd.DataFrame(columns=["종목","시세","전일비","등락율","시가총액","거래량"])
    
#    for company in companies:
#        df.loc[len(df)] = {
#            "종목":company["nm"],
#            "시세":company["nv"],
#            "전일비":company["cv"],
#            "등락율":company["cr"],
#            "시가총액":company["mks"],
#            "거래량":company["aq"],
#        }
#    return df
    
    print(response)
    print(response.text)
    print(type(json_info))
    print(json_info)
    print(type(companies))
    print(companies)

In [60]:
url = make_url(1500,1)
df = get_data(url)
df.tail()

,aa,aq,cd,cr,cv,mks,mt,nm,nv,pcv,rf
1453,79,8033,284990,-0.10,-10,22,0,KBSTAR 200에너지화학,9875,9885,5
1454,2,225,284980,-1.78,-175,21,0,KBSTAR 200금융,9680,9855,5
1455,0,50,295820,-0.80,-80,15,0,ARIRANG 200동일가중,9940,10020,5
1456,280,28489,295860,-1.11,-110,15,0,ARIRANG 200모멘텀,9770,9880,5
1457,278,28667,295890,-1.23,-120,14,0,ARIRANG 200로우볼,9665,9785,5


##### dark sky api
- 위도와 경도를 입력하면 날씨 정보를 보내주는 api
- https://api.darksky.net/

In [30]:
FORECAST_TOKEN = "0ecb7ff9cb44d5b0b584844e928d171a"

In [38]:
def forecast(lat, lng):
    url = "https://api.darksky.net/forecast/{}/{},{}".format(FORECAST_TOKEN, lat, lng)
    response = requests.get(url)
    json_info = response.json()
    return json_info["timezone"], json_info["hourly"]["summary"]

In [39]:
lat = 37.512
lng = 126.954
forecast(lat, lng)

('Asia/Seoul', 'Mostly cloudy throughout the day.')

In [42]:
def forecast2(lat, lng):
    forecast = forecastio.load_forecast(FORECAST_TOKEN, lat, lng)
    byHourly = forecast.hourly()
    return byHourly.summary

In [43]:
lat = 37.512
lng = 126.954
forecast2(lat, lng)

'Mostly cloudy throughout the day.'

##### bs4
- 네이버 키워드 랭킹 데이터 크롤링
- 다음 키워드 랭킹 데이터 크롤링

In [80]:
# dom.select - 여러 개의 html 엘리먼트를 셀렉팅 할 때 사용, 결과로 리스트 데이터를 리턴
# dom.select_one - 하나의 html 엘리먼트를 셀렉팅 할 때 사용, 결과로 문자열 데이터를 리턴

def naver():
    df = pd.DataFrame(columns = ["rank", "keyword"])
    
    response = requests.get("https://www.naver.com/")
    dom = BeautifulSoup(response.content, "html.parser") # 이 결과 dom 객체 형태로 나오기 때문에 dom이라고 변수명을 지정해준 거
    keywords = dom.select(".ah_roll_area > .ah_l > .ah_item") # ".ah_roll_area. PM_CL_realtimeKeyword_rolling" 하면 class 두개 select
    # 이 keywords는 ah_item 20개의 집합
#    print(len(keywords)) # length먼저 체크. 이런 식으로 찾아나가면 됨. 
    
    
    for keyword in keywords: # keywords에서 keyword 가져오는 거.
        df.loc[len(df)] = {
            "rank": keyword.select_one(".ah_r").text,
            "keyword": keyword.select_one(".ah_k").text,
        }
    
    return df 

naver_df = naver()
naver_df["keyword"]

0          이집트 우루과이
1              뮤직뱅크
2              블랙핑크
3              광주날씨
4             포항 약국
5                쌈디
6            모로코 이란
7                살라
8     러시아 사우디 하이라이트
9               민갑룡
10              강혁민
11              심영은
12              ksl
13              박기량
14        한국예술인복지재단
15         포르투갈 스페인
16             경찰청장
17            국가장학금
18           프로듀스48
19            하트시그널
Name: keyword, dtype: object

In [52]:
ls = []
for idx in range(3):
    print(idx)
    ls.append(naver())
    time.sleep(5) #한시간 하고 싶으면 3600이라 쓰는 것 보다 60*60. 2시간 하고 싶으면 60*60*2 이런 식이 가독성이 좋음

0
1
2


In [53]:
ls[0]

,rank,keyword
0,1,조현배
1,2,이집트 우루과이
2,3,쌈디
3,4,민갑룡
4,5,살라
5,6,포항 약국
6,7,심영은
7,8,경찰청장
8,9,장윤정 엄마
9,10,러시아 사우디 하이라이트


In [54]:
ls[1]

,rank,keyword
0,1,조현배
1,2,이집트 우루과이
2,3,쌈디
3,4,민갑룡
4,5,살라
5,6,포항 약국
6,7,심영은
7,8,경찰청장
8,9,장윤정 엄마
9,10,러시아 사우디 하이라이트


In [56]:
ls[2]

,rank,keyword
0,1,조현배
1,2,이집트 우루과이
2,3,쌈디
3,4,민갑룡
4,5,살라
5,6,포항 약국
6,7,심영은
7,8,경찰청장
8,9,장윤정 엄마
9,10,러시아 사우디 하이라이트


In [ ]:
# 다음의 실시간 이슈 검색어 10개 수집
# 네이버 20개 데이터와 다음 10개 데이터에서 중복되는 데이터를 출력 

In [71]:
def daum():
    df = pd.DataFrame(columns = ["rank", "keyword"])
    
    response = requests.get("https://www.daum.net/")
    dom = BeautifulSoup(response.content, "html.parser") # 이 결과 dom 객체 형태로 나오기 때문에 dom이라고 변수명을 지정해준 거
    keywords = dom.select(".roll_txt") # ".ah_roll_area. PM_CL_realtimeKeyword_rolling" 하면 class 두개 select
    # 이 keywords는 ah_item 20개의 집합
    print(len(keywords)) # length먼저 체크. 이런 식으로 찾아나가면 됨. 
    
    for keyword in keywords: # keywords에서 keyword 가져오는 거.
        df.loc[len(df)] = {
            "rank": keyword.select_one(".ir_wa").text,
            "keyword": keyword.select_one(".txt_issue").text,
        }    

    return df 

daum()

30


,rank,keyword
0,1위,민갑룡
1,2위,경찰청장
2,3위,국가장학금
3,4위,한국장학재단
4,5위,조현배
5,6위,블랙핑크
6,7위,안희정
7,8위,박원빈
8,9위,심영은
9,10위,김무성


In [85]:
# 강사님 거

def daum():
    df = pd.DataFrame()
    rank = []
    keywords = []
    response = requests.get("https://www.daum.net/")
    dom = BeautifulSoup(response.content, 'html.parser')
    keywords = dom.select(".realtime_part .list_hotissue.issue_row > li")
#    print(len(keywords))
    rank = [keyword.select_one(".ir_wa").text[:-1] for keyword in keywords]
#    print(rank)
    keywords = [keyword.select_one(".link_issue").text for keyword in keywords]
    df["rank"] = rank
    df["keyword"] = keywords
    return df
    
daum_df = daum()
daum_df["keyword"]

0       민갑룡
1      경찰청장
2     국가장학금
3    한국장학재단
4       조현배
5      블랙핑크
6       심영은
7       안희정
8       김무성
9       이찬오
Name: keyword, dtype: object

In [90]:
# naver_df["keyword"], daum_df["keyword"]

result = [
    keyword 
    for keyword in naver_df["keyword"] 
    if daum_df["keyword"].str.contains(keyword).any()
]

result

# set으로 교집합 만들어서 할 수도 있음

['블랙핑크', '민갑룡', '심영은', '경찰청장', '국가장학금']

In [88]:
naver_df["keyword"].str.contains("쌈디").any()

True

##### file down load: requests
- file의 url을 받아서 해당 url의 file을 다운로드 하는 방법

In [98]:
def download(title, download_link):
    response = requests.get(download_link, stream=True) #파일다운이라서 stream=True
    file_size = 0
    with open(title, 'wb') as f: #저장할 파일 열어줌
        for chunk in response.iter_content(chunk_size=1024): #1024의 단위는 byte. chunk라는 개념을 이용해서 잘라서 저장해주는 거
            if chunk:
                file_size += 1024
                f.write(chunk)
    return file_size

#if 100메가 짜리 파일을 한꺼번에 전송하다가 문제가 생겨서 특정 영역이 깨짐. 그러면 100메가를 처음부터 다시 보냄. 효율떨어져.
# 근데 이렇게 chunk로 잘라서 보내면? 중간에 깨져도 2,3메가만 더 보내면 되니까 전체를 보내는 것보다는 나음.

In [100]:
# url = "https://r4---sn-q4flrnez.googlevideo.com/videoplayback?id=o-AKTlJa0RFz51N8H0NrNwr2P2u5wj82b6MlO-ruQGKI8Z&itag=22&c=WEB&ip=23.239.195.119&key=yt6&ratebypass=yes&dur=34.342&lmt=1529039497166480&signature=A1030624D37944E6D27215D660C8E833EC79D676.70F3E7E206419745C3E0C4BF0DEA5D24BAEA0A75&ms=au%2Crdu&ei=NnojW-DwAYPTD769jWA&mv=u&mt=1529051193&expire=1529073302&pl=20&requiressl=yes&source=youtube&mime=video%2Fmp4&fvip=4&ipbits=0&mm=31%2C29&mn=sn-q4flrnez%2Csn-q4fl6nly&sparams=dur%2Cei%2Cid%2Cip%2Cipbits%2Citag%2Clmt%2Cmime%2Cmm%2Cmn%2Cms%2Cmv%2Cpl%2Cratebypass%2Crequiressl%2Csource%2Cexpire&video_id=ShQZhdjqjsw&title=iPhone+X+%E2%80%94+How+to+shoot+using+backlight+%E2%80%94+Apple
download_link = "https://bit.ly/2HRoZzl"
title = "video.mp4"
file_size = download(title, download_link)

In [102]:
str(round(file_size/1024/1024,2)) + "Mb"

'2.19Mb'